In [1]:
import os 

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'  # custom model folder
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8' # name of the model that we'll download from tf zoo
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('TensorFlow', 'Workspace'),
    'SCRIPTS_PATH': os.path.join('TensorFlow','scripts'),
    'APIMODEL_PATH': os.path.join('TensorFlow','models'),
    'ANNOTATION_PATH': os.path.join('TensorFlow', 'Workspace','annotations'),
    'IMAGE_PATH': os.path.join('TensorFlow', 'Workspace','Images'),
    'MODEL_PATH': os.path.join('TensorFlow', 'Workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('TensorFlow', 'Workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('TensorFlow', 'Workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('TensorFlow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('TensorFlow', 'Workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [6]:
import object_detection

#### Downloading our model from tf zoo

#### Create Label map 

In [7]:
labels = [{'name':'ThumbsUp', 'id':1}, {'name':'ThumbsDown', 'id':2}, {'name':'ThankYou', 'id':3}, {'name':'LiveLong', 'id':4}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')
        
# write labelmap as labels in annotations_path folder 
# labels must have same name as we did in labelimg

#### Create Tf record

In [8]:
# tfRecords are a binary file format for storing data 
# using a tfRecord help speedup training for your custom object detection model 

In [9]:
# it converts our annotations(xml) and images into fileformat that it can use 

In [10]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: TensorFlow/Workspace/annotations/train.record
Successfully created the TFRecord file: TensorFlow/Workspace/annotations/test.record


In [11]:
# Now we have train and test tfrecords that we'll use in our model

#### Copy Model Config to Training Folder

In [12]:
# Pipeline.config defines the architecture of the model

In [13]:
# copies pipeline.config to my_ssd_mobnet folder

In [14]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

#### Update Config For Transfer Learning

In [15]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [16]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [17]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [18]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [19]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [20]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [21]:
# we've changed the number of classes
# set our own fine tune checkpoint, labelmap path, input path, 

#### Things we have done so far

In [22]:
# we've steup all our paths
# downloaded and installed tf object detection api
# downloaded pretrained model from tensorflow model zoo
# created out label map 
# created tf records (train and test file)
# copied our config and updated that 

#### Train the model

In [23]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [24]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [25]:
print(command)

python TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=TensorFlow/Workspace/models/my_ssd_mobnet --pipeline_config_path=TensorFlow/Workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=2000


In [26]:
# (python TensorFlow/models/research/object_detection/model_main_tf2.py) it's running model_main_tf2.py located inside TensorFlow/models/research/object_detection/
# then we're passing number of arguments
# (--model_dir=TensorFlow/Workspace/models/my_ssd_mobnet) it's the model directory where pipeline.config file is 
# (--pipeline_config_path=TensorFlow/Workspace/models/my_ssd_mobnet/pipeline.config) passing pipeline.config
# (--num_train_steps=2000) number of steps

#### Evaluate the model

In [27]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [28]:
print(command)

python TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=TensorFlow/Workspace/models/my_ssd_mobnet --pipeline_config_path=TensorFlow/Workspace/models/my_ssd_mobnet/pipeline.config --checkpoint_dir=TensorFlow/Workspace/models/my_ssd_mobnet


In [29]:
# we can visualize loss and eval matrices using tensorboard located inside
# TensorFlow/Workspace/models/my_ssd_mobnet/train (loss matrix)
# TensorFlow/Workspace/models/my_ssd_mobnet/eval (eval matrix)

# tensorboard --logdir=.